![terrainbento logo](../images/terrainbento_logo.png)


# terrainbento Model BasicVs: Coupled Steady-State Solution

This model shows example usage of the Basic model from the TerrainBento package.

The BasicVs model implements modifies Basic to use variable source area runoff using the ''effective area'' approach:

$\frac{\partial \eta}{\partial t} = - KA_{eff}^{1/2}S + D\nabla^2 \eta$

where

$A_{eff} = A e^{-\alpha S / A}$

and  

$\alpha = \frac{K_{sat}  H_{init}  dx}{R_m}$.

Refer to [REF MANUSCRIPT] for the definition of all symbols.

This notebook (a) shows the initialization and running of this model, (b) saves a NetCDF file of the topography, which we will use to make an oblique Paraview image of the landscape, and (c) creates a slope-area plot at steady state.

In [ ]:
# import required modules
import os
import numpy as np

import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams.update({'font.size': 20})
%matplotlib inline

from landlab import imshow_grid
from landlab.io.netcdf import write_netcdf

from terrainbento import BasicVs

In [ ]:
# create the parameter dictionary needed to instantiate the model 
# all parameters used are specified in this notebook block

params = {# these three parameters are required by all models
          'dt' : 100, # years
          'output_interval': 1e4, # years
          'run_duration': 2e6, # years
          # these parameters are needed to specify the details of the model grid
          'number_of_node_rows' : 100,
          'number_of_node_columns' : 160,
          'node_spacing' : 10.0, # meters
          "add_random_noise": True, 
          "initial_noise_std": 1.,
          'random_seed': 4897, # set to initialize the topography with reproducible random noise
          # these parameter control geomorphic processes
          'water_erodability' : 0.001, # years^-1
          'm_sp' : 0.5, # unitless
          'n_sp' : 1.0, # unitless
          'regolith_transport_parameter' : 0.1, # meters^2/year
          'recharge_rate': 0.01, # meters/year
          'soil__initial_thickness' : 1.0, # meters
          'hydraulic_conductivity' : 15., # meters/year
          # these parameters control model boundary conditions
          "BoundaryHandlers": "NotCoreNodeBaselevelHandler",
          "NotCoreNodeBaselevelHandler": {"modify_core_nodes": True, "lowering_rate": -0.0005} , # meters/year
          # these parameter control how model output is written
          'save_first_timestep': True,
          'output_filename': 'output_netcdfs/basicVm.',
         }

In [ ]:
# initialize the model by passing the parameter dictionary
model = BasicVs(params=params)

# to run the model as specified by the parameters we execute the following line:
model.run(output_fields='topographic__elevation')

In [ ]:
# MAKE SLOPE-AREA PLOT

# plot nodes that are not on the boundary or adjacent to it
plotting_nodes = ((model.grid.node_x > params['node_spacing'])&
                  (model.grid.node_x < params['node_spacing']*(params['number_of_node_columns']-2))&
                  (model.grid.node_y > params['node_spacing'])&
                  (model.grid.node_y < params['node_spacing']*(params['number_of_node_rows']-2)))

# assign area_array and slope_array
area_array = model.grid.at_node['drainage_area'][plotting_nodes]
slope_array = model.grid.at_node['topographic__steepest_slope'][plotting_nodes]

# instantiate figure and plot
fig = plt.figure(figsize=(6, 3.75))
slope_area = plt.subplot()

# plot the data
slope_area.scatter(area_array, slope_array, marker='o', c='k', 
                   label = 'Model BasicVs') 

# make axes log and set limits
slope_area.set_xscale('log')
slope_area.set_yscale('log')

slope_area.set_xlim(9*10**1, 1*10**6)
slope_area.set_ylim(1e-4, 1e4)

# set x and y labels
slope_area.set_xlabel(r'Drainage area [m$^2$]')
slope_area.set_ylabel('Channel slope [-]')

slope_area.legend(scatterpoints=1,prop={'size':12})
slope_area.tick_params(axis='x', which='major', pad=7)

# save out an output figure
output_figure = os.path.join('output_figures/maintext_vsa_slope_area.eps')
fig.savefig(output_figure, bbox_inches='tight', dpi=1000) # save figure

In [ ]:
# Save stack of all netcdfs for Paraview to use.
model.save_to_xarray_dataset(filename="output_netcdfs/basicVs.nc", 
                             time_unit='years', 
                             reference_time='model start', 
                             space_unit='meters')
        
# remove temporary netcdfs
model.remove_output_netcdfs()

In [ ]:
# make a plot of the final steady state topography
imshow_grid(model.grid, 'topographic__elevation')

## Next Steps

- We recommend you review the [terrainbento manuscript](). **Link not yet active**

- There are three additional introductory tutorials: 

    1) [Introduction terrainbento](http://localhost:8888/notebooks/example_usage/Introduction_to_terrainbento.ipynb) 
    
    2) [Introduction to boundary conditions in terrainbento](http://localhost:8888/notebooks/example_usage/introduction_to_boundary_conditions.ipynb)
    
    3) **This Notebook**: [Introduction to OutputWriters in terrainbento](http://localhost:8888/notebooks/example_usage/introduction_to_output_writers.ipynb). 
    
    
- Five examples of steady state behavior in coupled process models can be found in the following notebooks:

    1) [Basic](http://localhost:8888/notebooks/coupled_process_elements/model_000_COUPLED_steady_solution.ipynb) the simplest landscape evolution model in the terrainbento package.

    2) [BasicVm](http://localhost:8888/notebooks/coupled_process_elements/model_001_COUPLED_steady_solution.ipynb) which permits the drainage area exponent to change.

    3) [BasicCh](http://localhost:8888/notebooks/coupled_process_elements/model_040_COUPLED_steady_solution.ipynb) which uses a non-linear hillslope erosion and transport law.

    4) [BasicVs](http://localhost:8888/notebooks/coupled_process_elements/model_200_COUPLED_steady_solution.ipynb) which uses variable source area hydrology.

    5) [BasisRt](http://localhost:8888/notebooks/coupled_process_elements/model_800_COUPLED_steady_solution.ipynb) which allows for two lithologies with different K values.